# Register Model

## Notebook Overview

- Start Execution
- Install and Import Libraries
- Configure Settings
- Log the Model to MLFlow
- Fetch the Latest Model Version from MLflow
- Load the Model and Run Inference

# Start Execution

In [1]:
import logging
import time

# Configure logger
logger: logging.Logger = logging.getLogger("register_model_logger")
logger.setLevel(logging.INFO)
logger.propagate = False  # Prevent duplicate logs from parent loggers

# Set formatter
formatter: logging.Formatter = logging.Formatter(
    fmt="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S"
)

# Configure and attach stream handler
stream_handler: logging.StreamHandler = logging.StreamHandler()
stream_handler.setFormatter(formatter)
logger.addHandler(stream_handler)

In [2]:
start_time = time.time()
logger.info("Notebook execution started.")

2025-08-12 16:08:56 - INFO - Notebook execution started.


# Install and Import Libraries

In [3]:
%%time

%pip install -r ../requirements.txt --quiet 

Note: you may need to restart the kernel to use updated packages.
CPU times: user 294 ms, sys: 103 ms, total: 397 ms
Wall time: 15.6 s


In [4]:
import os
import json
import sys
from pathlib import Path
from datetime import datetime
import warnings
import re

import pandas as pd
import numpy as np
from tabulate import tabulate

# MLflow for Experiment Tracking and Model Management
import mlflow
import mlflow.pyfunc
from mlflow import MlflowClient
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec, TensorSpec, ParamSchema, ParamSpec
from mlflow.tracking import MlflowClient

# # Define the relative path to the 'src' directory (two levels up from current working directory)
src_path = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add 'src' directory to system path for module imports (e.g., utils)
if src_path not in sys.path:
    sys.path.append(src_path)

from src.bert_recommendation_service import BERTTourismModel

import torch

/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/pyfunc/utils/data_validation.py:168: UserWarning: Add type hints to the `predict` method to enable data validation and automatic signature inference during model logging. Check https://mlflow.org/docs/latest/model/python_model.html#type-hint-usage-in-pythonmodel for more details.
  color_warning(


# Configure Settings

In [5]:
# ------------------------ Suppress Verbose Logs ------------------------
warnings.filterwarnings("ignore")

In [6]:
CORPUS_PATH = "../data/raw/corpus.csv"
TOKENIZER_DIR = "../artifacts/tokenizer"
BERT_MODEL_NAME = "bert-large-uncased"
BERT_MODEL_DATAFABRIC_PATH = "/home/jovyan/datafabric/Bertlargeuncased/bertlargeuncased.nemo"
EMBEDDINGS_OUTPUT_PATH = "../data/processed/"
BERT_MODEL_ONLINE_PATH = "/root/.cache/torch/NeMo/NeMo_1.22.0/bertlargeuncased/ca4ebba9f05a8ffb79845249ca046983/bertlargeuncased.nemo"
DEMO_PATH = "../demo"
EMBEDDINGS_PATH = "../data/processed/embeddings.csv"

# Define required constants for MLflow registration
EXPERIMENT_NAME = "BERT_Tourism_Experiment"
RUN_NAME = "BERT_Tourism_Run"
MODEL_NAME = "BERT_Tourism_Model"


# Register and Log the Model to MLFlow

In [7]:
%%time

mlflow.set_tracking_uri(os.getenv("MLFLOW_TRACKING_URI", "/phoenix/mlflow"))

# Set the MLflow experiment name
mlflow.set_experiment(experiment_name=EXPERIMENT_NAME)

logger.info(f'Starting the experiment: {EXPERIMENT_NAME}')
logger.info(f"Using MLflow tracking URI: {mlflow.get_tracking_uri()}")

# Start an MLflow run
with mlflow.start_run(run_name=RUN_NAME) as run:
    # Print the artifact URI for reference
    logging.info(f"Run's Artifact URI: {run.info.artifact_uri}")
    
    # Log the BERT similarity model to MLflow
    BERTTourismModel.log_model(
        model_name=MODEL_NAME,
        corpus_path=CORPUS_PATH,
        embeddings_path=EMBEDDINGS_PATH,
        tokenizer_dir=TOKENIZER_DIR,
        bert_model_online_path=BERT_MODEL_ONLINE_PATH,
        bert_model_datafabric_path=BERT_MODEL_DATAFABRIC_PATH,
        demo_path=DEMO_PATH,
        config_path="../configs/config.yaml"
    )
    # Register the logged model in MLflow Model Registry
    model_uri = f"runs:/{run.info.run_id}/{MODEL_NAME}"
    mlflow.register_model(model_uri=model_uri, name=MODEL_NAME)

logger.info(f"✅ Model '{MODEL_NAME}' successfully logged and registered under experiment '{EXPERIMENT_NAME}'.")

Traceback (most recent call last):
  File "/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 329, in search_experiments
    exp = self._get_experiment(exp_id, view_type)
  File "/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 427, in _get_experiment
    meta = FileStore._read_yaml(experiment_dir, FileStore.META_DATA_FILE_NAME)
  File "/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1373, in _read_yaml
    return _read_helper(root, file_name, attempts_remaining=retries)
  File "/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/store/tracking/file_store.py", line 1366, in _read_helper
    result = read_yaml(root, file_name)
  File "/opt/conda/envs/aistudio/lib/python3.10/site-packages/mlflow/utils/file_utils.py", line 310, in read_yaml
    raise MissingConfigException(f"Yaml file '{file_path}' does not exist.")
mlflow.exceptions.Missi

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

[NeMo W 2025-08-12 16:12:08 modelPT:617] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2025-08-12 16:12:08 modelPT:728] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 4.375e-05
        maximize: False
        weight_decay: 0.01
    )


[NeMo W 2025-08-12 16:12:08 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2025-08-12 16:12:10 save_restore_connector:249] Model BERTLMModel was successfully restored from /home/jovyan/datafabric/Bertlargeuncased/bertlargeuncased.nemo.


2025-08-12 16:12:10 - INFO - 🔧 Generating ONNX model(s) for specified models...
2025-08-12 16:12:10 - INFO - 🔄 Converting pytorch model: bert_tourism_onnx
2025-08-12 16:12:10 - INFO - 🔍 Model identified as: pytorch
2025-08-12 16:12:10 - INFO - 🔄 Exporting loaded PyTorch model with opset 12...
2025-08-12 16:14:14 - INFO - Model saved to ONNX: bert_tourism_onnx.onnx
2025-08-12 16:14:14 - INFO - ✅ PyTorch model exported to: bert_tourism_onnx.onnx
2025-08-12 16:14:15 - INFO - ✅ Converted bert_tourism_onnx: bert_tourism_onnx.onnx
2025-08-12 16:14:15 - INFO - 📦 Added ONNX artifact: onnx_model -> bert_tourism_onnx.onnx
2025-08-12 16:14:15 - INFO -   Creating individual model directories for artifacts...
2025-08-12 16:14:15 - INFO -   Copying ONNX models for 1 models: ['bert_tourism_onnx']
2025-08-12 16:14:15 - INFO - 📄 Model file already has correct name: bert_tourism_onnx.onnx
2025-08-12 16:14:15 - INFO - 📦 Added model file artifact: model_bert_tourism_onnx -> bert_tourism_onnx.onnx
2025-08-

2025-08-12 16:23:32 - INFO - Model logged with artifacts: ['corpus_path', 'embeddings_path', 'tokenizer_dir', 'bert_model_path', 'demo', 'config', 'onnx_model', 'model_bert_tourism_onnx']
2025-08-12 16:23:32 - INFO - ✅ Model logged with 1 model directories created!
Registered model 'BERT_Tourism_Model' already exists. Creating a new version of this model...
Created version '20' of model 'BERT_Tourism_Model'.
2025-08-12 16:23:40 - INFO - ✅ Model 'BERT_Tourism_Model' successfully logged and registered under experiment 'BERT_Tourism_Experiment'.


CPU times: user 2min 48s, sys: 3min 56s, total: 6min 44s
Wall time: 14min 14s


# Fetch the Latest Model Version from MLflow

In [8]:
# Initialize the MLflow client
client = MlflowClient()

# Retrieve the latest version of the "BERT_Tourism_Model" model (not yet in a specific stage)
versions = client.get_latest_versions(MODEL_NAME, stages=["None"])
if not versions:
    raise RuntimeError(f"No registered versions found for model '{MODEL_NAME}'.")
latest_version = versions[0].version

# Fetch model information, including its signature
model_info = mlflow.models.get_model_info(f"models:/{MODEL_NAME}/{latest_version}")

# Print the latest model version and its signature
print(f"Latest registered version of '{MODEL_NAME}': {latest_version}")
print(f"Signature: {model_info.signature}")

Latest registered version of 'BERT_Tourism_Model': 20
Signature: inputs: 
  ['query': string (required)]
outputs: 
  ['List of Pledges and Similarities': Tensor('object', (-1,))]
params: 
  ['show_score': boolean (default: False)]



# Load the Model and Run Inference

In [9]:
%%time

# Load the trained BERT similarity model from MLflow
model = mlflow.pyfunc.load_model(model_uri=f"models:/{MODEL_NAME}/{latest_version}")
print(f"Successfully loaded model '{MODEL_NAME}' version {latest_version} for inference.")

# Define a sample query for testing
query = "Give me a resort budget vacation suggestion"

# Use the model to predict similar results based on the query
result = model.predict({"query": [query]})

[NeMo W 2025-08-12 16:31:27 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    data_file: /home/yzhang/data/nlp/bert/47316/hdf5/lower_case_1_seq_len_512_max_pred_80_masked_lm_prob_0.15_random_seed_12345_dupe_factor_5_shard_1472_test_split_10/books_wiki_en_corpus/training/
    max_predictions_per_seq: 80
    batch_size: 16
    shuffle: true
    num_samples: -1
    num_workers: 2
    drop_last: false
    pin_memory: false
    
[NeMo W 2025-08-12 16:31:50 modelPT:617] Trainer wasn't specified in model constructor. Make sure that you really wanted it.


[NeMo I 2025-08-12 16:31:50 modelPT:728] Optimizer config = AdamW (
    Parameter Group 0
        amsgrad: False
        betas: (0.9, 0.999)
        capturable: False
        differentiable: False
        eps: 1e-08
        foreach: None
        fused: None
        lr: 4.375e-05
        maximize: False
        weight_decay: 0.01
    )


[NeMo W 2025-08-12 16:31:50 lr_scheduler:890] Neither `max_steps` nor `iters_per_batch` were provided to `optim.sched`, cannot compute effective `max_steps` !
    Scheduler will not be instantiated !


[NeMo I 2025-08-12 16:32:04 save_restore_connector:249] Model BERTLMModel was successfully restored from /phoenix/mlflow/569123593767764897/5e6cc3e750b34eb18dd6226072ddfe78/artifacts/BERT_Tourism_Model/artifacts/bertlargeuncased.nemo.


2025-08-12 16:32:05 - INFO - PyFunc Corpus total lines: 10374
2025-08-12 16:32:05 - INFO - PyFunc Embeddings total lines: 10374
2025-08-12 16:32:05 - INFO - === DEBUG CORPUS PYFUNC ===
2025-08-12 16:32:05 - INFO - Corpus shape: (10374, 3)
2025-08-12 16:32:05 - INFO - Embeddings shape: (10374, 1024)
2025-08-12 16:32:05 - INFO - First 3 texts:
2025-08-12 16:32:05 - INFO -   0: Actually we as an association are still pretty much at the beginning due to the pandemic which took the better part of our ressources. What we want to provide is a proper guideline for STR how to achieve, maintain and develop a sustainable business.
Additionally our business is very fragmented and diverse. We have privat hosts with one or only a few apartments, professional hosts, local property managers of all sizes, platforms for STR (local, national and international level) and service providers for the industry of all kind.
Our target for 2025 is to achieve that guideline. 

Nonetheless Some of our members have

Successfully loaded model 'BERT_Tourism_Model' version 20 for inference.


2025-08-12 16:32:05 - INFO - PyFunc input_ids shape: torch.Size([1, 9])
2025-08-12 16:32:05 - INFO - PyFunc input_ids[:10]: tensor([  101,  2507,  2033,  1037,  7001,  5166, 10885, 10293,   102])
2025-08-12 16:32:05 - INFO - PyFunc attention_mask[:10]: tensor([1, 1, 1, 1, 1, 1, 1, 1, 1])
2025-08-12 16:32:05 - INFO - PyFunc token_type_ids[:10]: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0])
2025-08-12 16:32:06 - INFO - PyFunc embedding shape: (1, 1024)
2025-08-12 16:32:06 - INFO - PyFunc embedding[:5]: [ 0.06976533  0.07480656 -0.891693    0.00676205 -0.21886505]
2025-08-12 16:32:06 - INFO - PyFunc embedding range: [-6.812421, 1.474453]
2025-08-12 16:32:06 - INFO - --------------------------------------------------


CPU times: user 2min 49s, sys: 1min 10s, total: 3min 59s
Wall time: 8min 15s


In [10]:
# Convert the result into a pandas DataFrame
df = pd.DataFrame(result)

# Drop unnecessary columns if needed
df = df.drop(columns=["Unnamed: 0", "Topic"], errors="ignore")

# Rename columns for better readability
df.rename(columns={"Pledge": "Recommended Option", "Similarity": "Relevance Score"}, inplace=True)

# Display the DataFrame in a tabular format
print(tabulate(df, headers="keys", tablefmt="fancy_grid"))

╒════╤═════════════════════════════════════════════════════════════════════════════════════════════════════╤═══════════════════╕
│    │ Recommended Option                                                                                  │   Relevance Score │
╞════╪═════════════════════════════════════════════════════════════════════════════════════════════════════╪═══════════════════╡
│  0 │ For a budget-friendly vacation, consider a resort with vacation options and cruise activities.      │          0.869162 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  1 │ For a budget-friendly vacation, consider a getaway with beach options and vacation activities.      │          0.863917 │
├────┼─────────────────────────────────────────────────────────────────────────────────────────────────────┼───────────────────┤
│  2 │ For a budget-friendly vacation, consider a getaway with hotel options and vacation activit

In [11]:
end_time: float = time.time()
elapsed_time: float = end_time - start_time
elapsed_minutes: int = int(elapsed_time // 60)
elapsed_seconds: float = elapsed_time % 60

logger.info(f"⏱️ Total execution time: {elapsed_minutes}m {elapsed_seconds:.2f}s")
logger.info("✅ Notebook execution completed successfully.")

2025-08-12 16:32:07 - INFO - ⏱️ Total execution time: 23m 10.95s
2025-08-12 16:32:07 - INFO - ✅ Notebook execution completed successfully.


Built with ❤️ using Z by HP AI Studio.